# Implementing the Spiking SSN simulation from Hennequin '18

In [ ]:
# Installation
using Revise
using LinearAlgebra,Statistics,StatsBase,Distributions
using Plots,NamedColors ; theme(:default)
using FFTW
using ProgressMeter
using Random
Random.seed!(0)
using NamedColors
col_exc = colorant"dark blue"
col_inh = colorant"dark red"

In [ ]:
# Initialization
N_E = 4000
N_I = 1000
p_E = 0.1
p_I = 0.4

v_rest = -70
tau_syn = 2*1E-3 # synaptic time constant
delta = 0.5*1E-3 # useful for parallelizing (Hennequin '18), code not parallelized at the moment

tau_E = 20*1E-3
tau_I = 10*1E-3
w = [1.25 -0.65
     1.2 -0.5]
alpha = 0.3
n = 2


noise = 0 # ignored for now
time_step = 2*1E-3

In [ ]:
# Input-Output function : r(V) = alpha*( V - V_rest )^n
function rate_powerlaw(v::Real)
    diff = v - v_rest
    diff = (diff < 0) ? 0 : diff
    return alpha*(diff^n)
end

function plot_powerlaw()
    count = 81
    v_arr = zeros(count)
    v_arr[1] = -90
    for i in 2:count
        v_arr[i] = v_arr[i-1] +1
    end
    rates = rate_powerlaw.(v_arr)
    plot(v_arr, rates, xlabel="Voltage (mV)", ylabel="Rate (Hz)", color="black", legend=false, fmt=:png)
end

plot_powerlaw()

In [ ]:
rate_powerlaw(-67)

In [ ]:
function numerical_rate(train::Vector{Float64})      # adopted from HawkesSimulator.jl
    if length(train) == 0
        return 0
    end
  Δt = train[end]-train[1]
  return length(train)/Δt
end

In [ ]:
# generic functions to plot the spike count

function plot_count(points)
    y = collect(1:length(points))
    plt = plot(xlabel="time (s)", ylabel="count", legend=:bottomright, fmt=:png)
    plot!(plt, points, y, label = "N(t)")
end

function plot_count(points_E, points_I)
    y = collect(1:length(points_E))
    plt = plot(xlabel="time (s)", ylabel="count", legend=:bottomright, fmt=:png)
    plot!(plt, points_E, y, label = "N_E(t)")
    y = collect(1:length(points_I))
    plot!(plt, points_I, y, label = "N_I(t)")
end

function plot_count(points_E, points_I, remean, rimean, count)
    y = collect(1:count)
    y1 = zeros(count)
    y2 = zeros(count)
    for i in 1:count
        y1[i] = remean*points_E[i]
        y2[i] = rimean*points_I[i]
    end
    plt = plot(xlabel="time (s)", ylabel="Spike Count", legend=:bottomright, fmt=:png)
    plot!(plt, points_E[1:count], y, label = "N_E(t)", color="blue")
    plot!(plt, points_I[1:count], y, label = "N_I(t)", color="red")
    plot!(plt, points_E[1:count],y1, label="E[N_E(t)]", color="dark blue")
    plot!(plt, points_I[1:count],y2, label="E[N_I(t)]", color="dark red")
end

In [ ]:
function spiking(probability)
    return rand(Float64) < probability
end

In [ ]:
# returns the effect of a spike at t_prevspike on spiking at current time t
@inline function interaction_kernel(t::Real, t_prevspike::Real, tau_kernel::Real)
    return t < zero(Real) ? zero(Real) : exp((t_prevspike+delta-t)/tau_kernel) / tau_kernel
end

In [ ]:
interaction_kernel(5,1,20)

In [ ]:
# returns the collective effect of all spikes in a spike train (spike_times) on spiking at current time t
function da_ssn(a_i::Real, t::Real, spike_times::Vector{Float64}, kind::Int64)
    num_spikes = size(spike_times, 1)
    tau = tau_E
    if kind == 2
        tau = tau_I
    end
    da = -1*a_i/tau_syn
    temp = 0.0
    for i in num_spikes:-1:1
        temp = interaction_kernel(t,spike_times[i], tau)
        da += temp
        if(temp<0.0001)
            break
        end
    end
    return da*time_step
end

In [ ]:
da_ssn(0, 3, [1,1.5,2,2.5, 2.9,2.99], 1)

In [ ]:
# Euler Method for a 1E-1I network
function dv_ssn_2d(v_i::Real, h_i::Real, noise_i::Real, tau_i::Real, j_arr::Vector{Float64}, a_arr::Vector{Float64})
    num_neurons = size(j_arr, 1)
    dv = v_rest - v_i + h_i + noise_i
    for i in 1:num_neurons
        dv += j_arr[i]*a_arr[i]
    end
    return dv*time_step/tau_i
end

In [ ]:
dv_ssn_2d(-70, 0, 0, 5, [2.0,3.0], [1.0,2.0])

In [ ]:
# simulating a 1 E-I network where the spikings are fed directly into the dv, using Eqs. 10 & 11 from Hennequin '18

function simulate_2d!(h, num_steps, t_arr, v_excite, v_inhibit, a, rate, spike_trainE, spike_trainI)
    
    v_excite[1] = v_rest
    v_inhibit[1] = v_rest
    
    @showprogress 1.0 "Running Spiking SSN simulation..." for i in 2:num_steps
        
        t_arr[i] = t_arr[i-1] + time_step
        a[i,:] = [a[i-1,1]+da_ssn(a[i-1,1], t_arr[i], spike_trainE, 1), a[i-1,2]+da_ssn(a[i-1,2], t_arr[i], spike_trainI, 2)]
        v_excite[i] = v_excite[i-1] + dv_ssn_2d(v_excite[i-1], h, noise, tau_E, [j[1,1], j[1,2]], a[i,:])
        v_inhibit[i] = v_inhibit[i-1] + dv_ssn_2d(v_inhibit[i-1], h, noise, tau_I, [j[2,1], j[2,2]], a[i,:])
        rate[i,:] = [rate_powerlaw(v_excite[i-1]), rate_powerlaw(v_inhibit[i-1])]
        spike_probability = rate[i,:] .* (time_step)
        if spiking(spike_probability[1])
            push!(spike_trainE, t_arr[i])
        end
        if spiking(spike_probability[2])
            push!(spike_trainI, t_arr[i])
        end
        
    end
    
end

In [ ]:
j = w
h = 5
t_max = 20
num_steps = Int(ceil(t_max/time_step))
t_arr = zeros(num_steps)
v_excite = zeros(num_steps)
v_inhibit = zeros(num_steps)
a = zeros(num_steps, 2)  # stored just for sanity check, implementation can be modified to improve space complexity
rate = zeros(num_steps, 2)
spike_trainE = Vector{Float64}()
spike_trainI = Vector{Float64}()

simulate_2d!(h, num_steps, t_arr, v_excite, v_inhibit, a, rate, spike_trainE, spike_trainI)

In [ ]:
rate_E = mean(rate[:,1])
rate_I = mean(rate[:,2])
println([rate_E, rate_I])

In [ ]:
numrate_E = numerical_rate(spike_trainE)
numrate_I = numerical_rate(spike_trainI)
println([numrate_E, numrate_I])

In [ ]:
v_emean = mean(v_excite)
v_imean = mean(v_inhibit)
println(v_emean)
println(v_imean)
println([rate_powerlaw(v_emean), rate_powerlaw(v_imean)])

In [ ]:
plot(t_arr[1:500], rate[1:500,:], xlabel="time (s)", ylabel="Rate (Hz)", label=["rate_e" "rate_i"], legend=:bottomright, fmt=:png)

In [ ]:
plt = plot(xlabel="time (s)", ylabel="Voltage (mV)", legend=:bottomright, fmt=:png)
plot!(plt, t_arr , v_excite, label = "V_excite")
plot!(plt, t_arr , v_inhibit, label = "V_inhibit")

In [ ]:
plot_count(spike_trainE, spike_trainI)

Extending the code to run for a large network with N_E excitatory and N_I inhibitory neurons

In [ ]:
# Each neuron will have:
# tau, J vector, a(t), rate(t), V(t), spike_train     (for now, all E have same tau, all I have same tau initialized globally)

struct neuron
    spike_train::Vector{Float64}
    v::Vector{Float64}
    rate::Vector{Float64}
    a::Vector{Float64} # stored just for sanity check, implementation can be modified to improve space complexity
    j_e::Vector{Bool}
    j_i::Vector{Bool}
end

In [ ]:
function create_j(x::Int64, y::Int64)
    # x is main neuron, considering all connections y->x
    # creating p*n random connections
    # j is a boolean here, the corresponding weight matrix (w/(tau_syn*p_E*p_I)) is calculated in dv_ssn
    n = N_E
    p = p_E
    if y == 2
        n = N_I
        p = p_I
    end
    j = falses(n)
    count = p*n
    for i in 1:count
        index = rand(1:n)
        while j[index] == true
            index = rand(1:n)
        end
        j[index] = true
    end
    return j
end

In [ ]:
function initialize_network!(excitatory_neuron::Vector{neuron}, inhibitory_neuron::Vector{neuron})
    
    @showprogress 1.0 "initializing E neurons..." for i in 1:N_E
        spike_train = Vector{Float64}()
        v = zeros(num_steps)
        v[1] = v_rest
        rate = zeros(num_steps)
        a = zeros(num_steps)
        j_e = create_j(1,1)
        j_i = create_j(1,2)
        neuron_i = neuron(spike_train, v, rate, a, j_e, j_i)

        push!(excitatory_neuron, neuron_i)
    end
    
    @showprogress 1.0 "initializing I neurons..." for i in 1:N_I
        spike_train = Vector{Float64}()
        v = zeros(num_steps)
        v[1] = v_rest
        rate = zeros(num_steps)
        a = zeros(num_steps)
        j_e = create_j(2,1)
        j_i = create_j(2,2)
        neuron_i = neuron(spike_train, v, rate, a, j_e, j_i)

        push!(inhibitory_neuron, neuron_i)
    end
end

In [ ]:
function update_a!(t::Real, neuron_i::neuron, i::Int64, kind::Int64)
    neuron_i.a[i] = neuron_i.a[i-1]+da_ssn(neuron_i.a[i-1], t, neuron_i.spike_train, kind)
end

In [ ]:
# Euler Method
function dv_ssn(v_i::Real, h_i::Real, noise_i::Real, kind::Int64, j_e::Vector{Bool}, j_i::Vector{Bool}, i::Int64)
    dv = v_rest - v_i + h_i + noise_i
    j = w[kind, 1]/(tau_syn*p_E*N_E)
    for index in 1:N_E
        dv += j*j_e[index]*excitatory_neuron[index].a[i]
    end
    j = w[kind, 2]/(tau_syn*p_I*N_I)
    for index in 1:N_I
        dv += j*j_i[index]*inhibitory_neuron[index].a[i]
    end
    tau = tau_E
    if kind == 2
        tau = tau_I
    end
    return dv*time_step/tau
end

In [ ]:
function ssn_step!(t::Real, h::Real, neuron_i::neuron, i::Int64, kind::Int64)
    
    neuron_i.v[i] = neuron_i.v[i-1] + dv_ssn(neuron_i.v[i-1], h, noise, kind, neuron_i.j_e, neuron_i.j_i, i)
    neuron_i.rate[i] = rate_powerlaw(neuron_i.v[i])
    spike_probability = neuron_i.rate[i] * time_step
    if spiking(spike_probability)
        push!(neuron_i.spike_train, t)
    end
    
end

In [ ]:
function simulate!(excitatory_neuron::Vector{neuron}, inhibitory_neuron::Vector{neuron})
    
    h = 5
    t_arr = zeros(num_steps)
    
    @showprogress 1.0 "Running Spiking SSN simulation..." for i in 2:num_steps
        
        t_arr[i] = t_arr[i-1] + time_step
        
        update_a!.(t_arr[i], excitatory_neuron, i, 1)
        
        update_a!.(t_arr[i], inhibitory_neuron, i, 2)
        
        ssn_step!.(t_arr[i], h, excitatory_neuron, i, 1)
        
        ssn_step!.(t_arr[i], h, inhibitory_neuron, i, 2)
        
#         for neuron_i in excitatory_neuron
#             update_a!(t_arr[i], neuron_i, i, 1)
#         end
#         for neuron_i in inhibitory_neuron
#             update_a!(t_arr[i], neuron_i, i, 2)
#         end
#         for neuron_i in excitatory_neuron
#             ssn_step!(t_arr[i], h, neuron_i, i, 1)
#         end
#         for neuron_i in inhibitory_neuron
#             ssn_step!(t_arr[i], h, neuron_i, i, 2)
#         end
    end
    
    return t_arr
end

In [ ]:
excitatory_neuron = Vector{neuron}()
inhibitory_neuron = Vector{neuron}()
t_max = 1   # running for 1 or 1.5 seconds is enough to get a good result
# can modify time_step to improve speed further, while keeping in mind all the time constant values
num_steps = Int(ceil(t_max/time_step))
initialize_network!(excitatory_neuron, inhibitory_neuron)

In [ ]:
t_arr = simulate!(excitatory_neuron, inhibitory_neuron)
println(num_steps)

In [ ]:
# saving the results of the large simulation for future reference
using JLD
save("/tmp/ssn_large.jld", "excitatory_neuron", excitatory_neuron, "inhibitory_neuron", inhibitory_neuron, "t_arr", t_arr)
save("/home/kapoorv/ssn_largedata.jld", "excitatory_neuron", excitatory_neuron, "inhibitory_neuron", inhibitory_neuron, "t_arr", t_arr)

In [ ]:
# checking the spiking rate of random E and I neurons
numrate_lE = numerical_rate(excitatory_neuron[2].spike_train)
numrate_lI = numerical_rate(inhibitory_neuron[1].spike_train)
println(numrate_lE)
println(numrate_lI)

In [ ]:
# checking the mean spiking rate of the entire E and entire I population
# should be equal to the steady state rates of a 2D network with w weight
numrates_E = zeros(N_E)
numrates_I = zeros(N_I)
for i in 1:N_E
    numrates_E[i] = numerical_rate(excitatory_neuron[i].spike_train)
end
for i in 1:N_I
    numrates_I[i] = numerical_rate(inhibitory_neuron[i].spike_train)
end

println(mean(numrates_E))
println(mean(numrates_I))

In [ ]:
# checking the mean voltages of the entire E and entire I population
# should be equal to the steady state voltages of a 2D network with w weight

mean_Ve = zeros(N_E)
mean_Vi = zeros(N_I)
for i in 1:N_E
    mean_Ve[i] = mean(excitatory_neuron[i].v)
end
for i in 1:N_I
    mean_Vi[i] = mean(inhibitory_neuron[i].v)
end
total_mean_Ve = mean(mean_Ve)
total_mean_Vi = mean(mean_Vi)
println([total_mean_Ve, total_mean_Vi])

# storing in vector to use for plotting
ve_arr = zeros(num_steps)
vi_arr = zeros(num_steps)
for i in 1:num_steps
    ve_arr[i] = total_mean_Ve
    vi_arr[i] = total_mean_Vi
end

In [ ]:
xe = collect(1:N_E)
plt = scatter(xlabel = "neuron number", fmt=:png)
scatter!(plt, xe, numrates_E, label = " numrates_E")

In [ ]:
xi = collect(1:N_I)
plt = scatter(xlabel = "neuron number", fmt=:png)
scatter!(plt, xi, numrates_I, label = " numrates_I")

In [ ]:
# checking the mean rates of the entire E and entire I population
# should be equal to the steady state rates of a 2D network with w weight = spiking rate
mean_ratesE = zeros(N_E)
mean_ratesI = zeros(N_I)
for i in 1:N_E
    mean_ratesE[i] = mean(excitatory_neuron[i].rate)
end

for i in 1:N_I
    mean_ratesI[i] = mean(inhibitory_neuron[i].rate)
end
total_mean_rateE = mean(mean_ratesE)
total_mean_rateI = mean(mean_ratesI)
re_arr = zeros(num_steps)
ri_arr = zeros(num_steps)
for i in 1:num_steps
    re_arr[i] = total_mean_rateE
    ri_arr[i] = total_mean_rateI
end
println(mean(mean_ratesE))
println(mean(mean_ratesI))

In [ ]:
xe = collect(1:N_E)
plt = scatter(xlabel = "neuron number", fmt=:png)
scatter!(plt, xe, mean_ratesE, label = "mean_ratesE")

In [ ]:
xi = collect(1:N_I)
plt = scatter(xlabel = "neuron number", fmt=:png)
scatter!(plt, xi, mean_ratesI, label = "mean_ratesI")

checking a random E and random I neuron

In [ ]:
plt = plot(xlabel="time (s)", ylabel = "Voltage (mV)", fmt=:png, legend=:bottomright)
plot!(plt, t_arr , excitatory_neuron[3000].v, label = "V_excite", color="blue")
plot!(plt, t_arr , inhibitory_neuron[100].v, label = "V_inhibit", color="red")
plot!(plt, t_arr, ve_arr, label = "E[V_excite]", color="dark blue", linestyle=:dash)
plot!(plt, t_arr, vi_arr, label = "E[V_inhibit]", color="dark red", linestyle=:dash)

In [ ]:
plt = plot(xlabel="time (s)", ylabel = "Rate (Hz)", fmt=:png, legend=:bottomright)
plot!(plt, t_arr , excitatory_neuron[3000].rate, label = "R_excite", color="blue")
plot!(plt, t_arr , inhibitory_neuron[100].rate, label = "R_inhibit", color="red")
plot!(plt, t_arr, re_arr, label = "E[R_excite]", color="dark blue", linestyle=:dash)
plot!(plt, t_arr, ri_arr, label = "E[R_inhibit]", color="dark red", linestyle=:dash)

In [ ]:
function rasterplot(tlims = (0.,1.) )
  _trainE = excitatory_neuron[3500].spike_train
  plt=plot()
  trainE = filter(t-> tlims[1]< t < tlims[2],_trainE)
  nspk = size(trainE,1)
  scatter!(plt,trainE,fill(4,nspk),markersize=25, markercolor=:blue,markershape=:vline,legend=:topright, label="Excitatory #3500")
  _trainE = excitatory_neuron[40].spike_train
  trainE = filter(t-> tlims[1]< t < tlims[2],_trainE)
  nspk = size(trainE,1)
  scatter!(plt,trainE,fill(3,nspk),markersize=25, markercolor=:blue,markershape=:vline,legend=:topright, label="Excitatory #40")
  _trainI = inhibitory_neuron[200].spike_train
    trainI = filter(t-> tlims[1]< t < tlims[2],_trainI)
  nspk = size(trainI,1)
  scatter!(plt,trainI,fill(2,nspk),markersize=25, markercolor=:red,markershape=:vline,legend=:topright, label="Inhibitory #200")
  _trainI = inhibitory_neuron[50].spike_train
    trainI = filter(t-> tlims[1]< t < tlims[2],_trainI)
  nspk = size(trainI,1)
  scatter!(plt,trainI,fill(1,nspk),markersize=25, markercolor=:red,markershape=:vline,legend=:topright, label="Inhibitory #50")
  plot!(plt,ylims=(0,6),xlabel="time (s)",fmt=:png)
end

rasterplot()

In [ ]:
plot_count(excitatory_neuron[3500].spike_train, inhibitory_neuron[50].spike_train, total_mean_rateE, total_mean_rateI, 11)

In [ ]:
# checking the mean rate of random neurons across different time ranges to find out a good t_max
# conclusion -> t_max = 1 second is good enough for the 4000 E - 1000 I network

In [ ]:
rate_1E = mean(excitatory_neuron[1500].rate[1:1000])
rate_2E = mean(excitatory_neuron[1500].rate[1:2000])
rate_3E = mean(excitatory_neuron[1500].rate[1:3000])
rate_4E = mean(excitatory_neuron[1500].rate[1:4000])
rate_5E = mean(excitatory_neuron[1500].rate[1:5000])
rate_6E = mean(excitatory_neuron[1500].rate[1:6000])
println([rate_1E, rate_2E, rate_3E, rate_4E, rate_5E, rate_6E])

In [ ]:
rate_1E = mean(excitatory_neuron[100].rate[1:1000])
rate_2E = mean(excitatory_neuron[100].rate[1:2000])
rate_3E = mean(excitatory_neuron[100].rate[1:3000])
rate_4E = mean(excitatory_neuron[100].rate[1:4000])
rate_5E = mean(excitatory_neuron[100].rate[1:5000])
rate_6E = mean(excitatory_neuron[100].rate[1:6000])
println([rate_1E, rate_2E, rate_3E, rate_4E, rate_5E, rate_6E])

In [ ]:
rate_1E = mean(excitatory_neuron[3670].rate[1:1000])
rate_2E = mean(excitatory_neuron[3670].rate[1:2000])
rate_3E = mean(excitatory_neuron[3670].rate[1:3000])
rate_4E = mean(excitatory_neuron[3670].rate[1:4000])
rate_5E = mean(excitatory_neuron[3670].rate[1:5000])
rate_6E = mean(excitatory_neuron[3670].rate[1:6000])
println([rate_1E, rate_2E, rate_3E, rate_4E, rate_5E, rate_6E])

In [ ]:
rate_1I = mean(inhibitory_neuron[900].rate[1:1000])
rate_2I = mean(inhibitory_neuron[900].rate[1:2000])
rate_3I = mean(inhibitory_neuron[900].rate[1:3000])
rate_4I = mean(inhibitory_neuron[900].rate[1:4000])
rate_5I = mean(inhibitory_neuron[900].rate[1:5000])
rate_6I = mean(inhibitory_neuron[900].rate[1:6000])
println([rate_1I, rate_2I, rate_3I, rate_4I, rate_5I, rate_6I])

In [ ]:
rate_1I = mean(inhibitory_neuron[90].rate[1:1000])
rate_2I = mean(inhibitory_neuron[90].rate[1:2000])
rate_3I = mean(inhibitory_neuron[90].rate[1:3000])
rate_4I = mean(inhibitory_neuron[90].rate[1:4000])
rate_5I = mean(inhibitory_neuron[90].rate[1:5000])
rate_6I = mean(inhibitory_neuron[900].rate[1:6000])
println([rate_1I, rate_2I, rate_3I, rate_4I, rate_5I, rate_6I])

In [ ]:
# Using Equations 2 & 3 from Hennequin '18
# i.e. the non spiking model
# INCOMPLETE

In [ ]:
function update_r!(t::Real, neuron_i::neuron, i::Int64, kind::Int64)
    neuron_i.rate[i] = rate_powerlaw(neuron_i.v[i-1])
end

In [ ]:
# modelling noise as a Multivariate Ornstein-Uhlenbeck process
function dnoise!(noise::Vector{Float64}, wiener::Vector{Float64}, cov_noise::Matrix{Float64})
    wiener = wiener + sqrt(time_step)*[rand(Normal(0,0.5)),rand(Normal(0,0.5))]
    dn = - noise .* time_step + sqrt(2*tau_noise*cov_noise)*wiener
    return dn ./ tau_noise
end

In [ ]:
# Euler Method
function dv_ssn(v_i::Real, h_i::Real, noise_i::Real, kind::Int64, j_e::Vector{Bool}, j_i::Vector{Bool}, i::Int64)
    dv = v_rest - v_i + h_i + noise_i
    j = w[kind, 1]/(tau_syn*p_E*N_E)    
    
    for index in 1:N_E
        dv += j*j_e[index]*excitatory_neuron[index].rate[i]
    end
    j = w[kind, 2]/(tau_syn*p_I*N_I)
    for index in 1:N_I
        dv += j*j_i[index]*inhibitory_neuron[index].rate[i]
    end
    tau = tau_E
    if kind == 2
        tau = tau_I
    end
    return dv*time_step/tau
end

In [ ]:
function ssn_step!(t::Real, h::Real, neuron_i::neuron, i::Int64, kind::Int64)
    neuron_i.v[i] = neuron_i.v[i-1] + dv_ssn(neuron_i.v[i-1], h, noise, kind, neuron_i.j_e, neuron_i.j_i, i)
   spike_probability = neuron_i.rate[i] * time_step
    if spiking(spike_probability)
        push!(neuron_i.spike_train, t)
    end     
end

In [ ]:
function simulate()
    
    h = 10
    t_arr = zeros(num_steps)
    
    @showprogress 1.0 "Running Spiking SSN simulation..." for i in 2:num_steps
        
        t_arr[i] = t_arr[i-1] + time_step
        for neuron_i in excitatory_neuron
            update_r!(t_arr[i], neuron_i, i, 1)
        end
        for neuron_i in inhibitory_neuron
            update_r!(t_arr[i], neuron_i, i, 2)
        end
        for neuron_i in excitatory_neuron
            ssn_step!(t_arr[i], h, neuron_i, i, 1)
        end
        for neuron_i in inhibitory_neuron
            ssn_step!(t_arr[i], h, neuron_i, i, 2)
        end
#         println("i = $(i) size of rate of excitatory[5] = ",size(excitatory_neuron[5].rate))
    end
    return t_arr
end

In [ ]:
excitatory_neuron = Vector{neuron}()
inhibitory_neuron = Vector{neuron}()
t_max = 5
num_steps = Int(ceil(t_max/time_step))
initialize_network!(excitatory_neuron, inhibitory_neuron)

In [ ]:
t_arr = simulate()

In [ ]:
excitatory_neuron[1].v[1]

In [ ]:
plt = plot(xlabel="time (s)")
plot!(plt, t_arr , excitatory_neuron[1].rate, label = "V_excite")
plot!(plt, t_arr , inhibitory_neuron[1].rate, label = "V_inhibit")